In [2]:
import pandas as pd
import time
import openai
import os

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL_NAME = "gpt-4"

In [ ]:

train_pos_data = pd.read_csv('../data/amdb_dataset/train/train_pos_data.csv')
train_neg_data = pd.read_csv('../data/amdb_dataset/train/train_neg_data.csv')
test_pos_data = pd.read_csv('../data/amdb_dataset/test/test_pos_data.csv')
test_neg_data = pd.read_csv('../data/amdb_dataset/test/test_neg_data.csv')

train_pos_data['true_label'] = 'P'
train_neg_data['true_label'] = 'N'
test_pos_data['true_label'] = 'P'
test_neg_data['true_label'] = 'N'

train_df = pd.concat([train_pos_data, train_neg_data], ignore_index=True)
test_df = pd.concat([test_pos_data, test_neg_data], ignore_index=True)
    
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)


train_df

,index,text,mark,true_label
0,6868,In Panic In The Streets Richard Widmark plays ...,8,P
1,24016,If you ask me the first one was really better ...,1,N
2,9668,I am a big fan a Faerie Tale Theatre and I've ...,10,P
3,13640,I just finished reading a book about Dillinger...,1,N
4,14018,Greg Davis and Bryan Daly take some crazed sta...,2,N
...,...,...,...,...
24995,21575,My roommates & I nearly shorted out our TV fro...,2,N
24996,5390,Michelle Rodriguez is the defining actress who...,7,P
24997,860,Nice movie with a great soundtrack which spans...,8,P
24998,15795,"Even though this was a made-for-TV production,...",1,N


In [ ]:
def classify_comment_with_prompt(comment, messages, model_name):
    start_time = time.time()
    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=messages,
            temperature=0
        )
        end_time = time.time()
        
        pred = response['choices'][0]['message']['content'].strip()
        elapsed_time = end_time - start_time
        
        prompt_tokens = response['usage']['prompt_tokens']
        completion_tokens = response['usage']['completion_tokens']
        
        return pred, elapsed_time, prompt_tokens, completion_tokens
    except Exception as e:
        print(f"Ошибка при обработке комментария: {comment[:50]}... Ошибка: {e}")
        return "error", 0, 0, 0

In [ ]:
prompts_dict = {
    "Friendly positive/negative": (
        "You are a friendly assistant. Classify the following comment as Positive or Negative. "
        "Now classify the comment:\nComment: {text}\nSentiment:"
    ),
    
    "Strict formal": (
        "You are a very strict and formal sentiment analyzer. Only label as Positive or Negative, no exceptions. "
        "Examples:\n"
        "'Fantastic service!' -> Positive\n"
        "'I am extremely disappointed.' -> Negative\n\n"
        "Classify the comment:\n{text}\nSentiment:"
    ),
    
    "Emotional tone": (
        "You are an emotional sentiment detector. Label the comment as Positive or Negative. "
        "Examples:\n"
        "'I feel so happy and grateful today!' -> Positive\n"
        "'I am sad and frustrated with this.' -> Negative\n\n"
        "Classify the comment:\n{text}\nSentiment:"
    ),
    
    "Direct short": (
        "Classify as Positive or Negative.\n"
        "Example: 'I hate this!' -> Negative\n"
        "Example: 'Love it!' -> Positive\n"
        "Comment: {text}\nSentiment:"
    )
}

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [ ]:

def evaluate_prompts(prompts_dict, data_df, sample_size=50, model_name=MODEL_NAME):
    results_list = []
    sample_df = data_df.sample(n=min(sample_size, len(data_df)), random_state=42).copy()
    
    for prompt_name, prompt_template in prompts_dict.items():
        sample_df[f'predicted_label_{prompt_name}'] = None
        sample_df[f'time_sec_{prompt_name}'] = None
        sample_df[f'prompt_tokens_{prompt_name}'] = None
        sample_df[f'completion_tokens_{prompt_name}'] = None

        all_preds = []
        all_times = []
        all_prompt_tokens = []
        all_completion_tokens = []

        for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc=f"Evaluating {prompt_name}"):
            comment = row['text']
            
            pred, elapsed_time, prompt_tokens, completion_tokens = classify_comment_with_prompt(
                comment, prompt_template, model_name
            )
            
            if "positive" in pred.lower():
                pred = "P"
            elif "negative" in pred.lower():
                pred = "N"
            else:
                pred = "None"

            all_preds.append(pred)
            all_times.append(elapsed_time)
            all_prompt_tokens.append(prompt_tokens)
            all_completion_tokens.append(completion_tokens)

        sample_df[f'predicted_label_{prompt_name}'] = all_preds
        sample_df[f'time_sec_{prompt_name}'] = all_times
        sample_df[f'prompt_tokens_{prompt_name}'] = all_prompt_tokens
        sample_df[f'completion_tokens_{prompt_name}'] = all_completion_tokens

        true_labels = sample_df['true_label'].tolist()
        filtered_true = [true_labels[i] for i, pred in enumerate(all_preds) if pred in ['P', 'N']]
        filtered_preds = [pred for pred in all_preds if pred in ['P', 'N']]

        if len(filtered_true) == 0:
            accuracy, precision, recall, f1 = 0, 0, 0, 0
        else:
            accuracy = accuracy_score(filtered_true, filtered_preds)
            precision = precision_score(filtered_true, filtered_preds, pos_label='P', zero_division=0)
            recall = recall_score(filtered_true, filtered_preds, pos_label='P', zero_division=0)
            f1 = f1_score(filtered_true, filtered_preds, pos_label='P', zero_division=0)

        avg_time = sum(all_times) / len(all_times) if all_times else 0

        results_list.append({
            'prompt_name': prompt_name,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'avg_time_per_comment': avg_time,
            'total_prompt_tokens': sum(all_prompt_tokens),
            'total_completion_tokens': sum(all_completion_tokens)
        })
    
    results_df = pd.DataFrame(results_list)
    return results_df, sample_df

In [ ]:
def classify_dataset(train_df, test_df, prompt, model_name=MODEL_NAME):
    def process_df(df, df_name):
        df = df.copy()
        df['predicted_label'] = None
        df['time_sec'] = None
        df['prompt_tokens'] = None
        df['completion_tokens'] = None

        all_preds, all_times, all_prompt_tokens, all_completion_tokens = [], [], [], []

        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {df_name}"):
            comment = row['text']

            pred, elapsed_time, prompt_tokens, completion_tokens = classify_comment_with_prompt(
                comment, prompt, model_name
            )

            if "positive" in pred.lower():
                pred = "P"
            elif "negative" in pred.lower():
                pred = "N"
            else:
                pred = "None"

            all_preds.append(pred)
            all_times.append(elapsed_time)
            all_prompt_tokens.append(prompt_tokens)
            all_completion_tokens.append(completion_tokens)

        df['predicted_label'] = all_preds
        df['time_sec'] = all_times
        df['prompt_tokens'] = all_prompt_tokens
        df['completion_tokens'] = all_completion_tokens

        metrics = {
            'avg_time_per_comment': sum(all_times)/len(all_times) if all_times else 0,
            'total_prompt_tokens': sum(all_prompt_tokens),
            'total_completion_tokens': sum(all_completion_tokens)
        }

        return df, metrics

    processed_train_df, train_metrics = process_df(train_df, "Train")
    processed_test_df, test_metrics = process_df(test_df, "Test")

    processed_train_df.to_csv("train_with_predictions.csv", index=False)
    processed_test_df.to_csv("test_with_predictions.csv", index=False)

    return processed_train_df, processed_test_df, train_metrics, test_metrics